In [1]:
#coding=utf-8
import sys 
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.metrics import  accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

data = pd.read_csv('E:\\备份\\训练数据汇总(8W)\\2019_4_15.csv')
data = data.loc[:, ~data.columns.str.contains('^Unnamed')]

name = ['PHONE_VERSION', 'VIDEO_CLARITY']
columns = data.columns.values.tolist()
name_1 = [name for index, name in enumerate(columns) if name != name[0] and name != name[1]]

scaler = MinMaxScaler()
X1 = data[name_1[0:16]]
X1_data = scaler.fit_transform(X1)
Y = data[name_1[16:21]]
X2 = data[name[0]]
X3 = data[name[1]]

X2_data = OneHotEncoder().fit_transform(X2.values.reshape(-1, 1)).todense().getA()
X3_data = OneHotEncoder().fit_transform(X3.values.reshape(-1, 1)).todense().getA()

X_data = np.hstack((X1_data, X2_data, X3_data))

Y1 = data[[name_1[16]]]
Y2 = data[[name_1[17]]]
Y3 = data[[name_1[18]]]
Y4 = data[[name_1[19]]]

[raw, col] = X_data.shape
#shape = 8 * 8
shape = 6 * 6
X4_data = np.zeros([raw, shape - col])
X_data = np.hstack((X_data, X4_data))

X1_train, X1_test, y1_train, y1_test = train_test_split(X_data, Y1, test_size=0.25, random_state = 33)
X2_train, X2_test, y2_train, y2_test = train_test_split(X_data, Y2, test_size=0.25, random_state = 33)
X3_train, X3_test, y3_train, y3_test = train_test_split(X_data, Y3, test_size=0.25, random_state = 33)
X4_train, X4_test, y4_train, y4_test = train_test_split(X_data, Y4, test_size=0.25, random_state = 33)

X1_train = X1_train.reshape(-1 , 6, 6, 1)
X1_test = X1_test.reshape(-1, 6 ,6 , 1)
y1_train = OneHotEncoder().fit_transform(y1_train.values.reshape(-1, 1)).todense().getA()
y1_test_code = OneHotEncoder().fit_transform(y1_test.values.reshape(-1, 1)).todense().getA()

X2_train = X2_train.reshape(-1,6,6,1)
X2_test = X2_test.reshape(-1,6,6,1)
y2_train = OneHotEncoder().fit_transform(y2_train.values.reshape(-1, 1)).todense().getA()
y2_test_code = OneHotEncoder().fit_transform(y2_test.values.reshape(-1, 1)).todense().getA()

X3_train = X3_train.reshape(-1,6,6,1)
X3_test = X3_test.reshape(-1,6,6,1)
y3_train = OneHotEncoder().fit_transform(y3_train.values.reshape(-1, 1)).todense().getA()
y3_test_code = OneHotEncoder().fit_transform(y3_test.values.reshape(-1, 1)).todense().getA()

X4_train = X4_train.reshape(-1,6,6,1)
X4_test = X4_test.reshape(-1,6,6,1)
y4_train = OneHotEncoder().fit_transform(y4_train.values.reshape(-1, 1)).todense().getA()
y4_test_code = OneHotEncoder().fit_transform(y4_test.values.reshape(-1, 1)).todense().getA()

C:\Users\ppxx\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\utils\fixes.py:313: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  _nan_object_mask = _nan_object_array != _nan_object_array
C:\Users\ppxx\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
C:\Users\ppxx\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\preprocessing\_encoders.py:363: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEnco

In [4]:
def generatebatch(X,Y,n_examples, batch_size): 
    for batch_i in range(n_examples // batch_size): 
        start = batch_i * batch_size 
        end = start + batch_size 
        batch_xs = X[start:end] 
        batch_ys = Y[start:end] 
        yield batch_xs, batch_ys 
        
# 使用MBGD算法，设定batch_size为8
batch_size = 1024
tf.reset_default_graph()

# 输入层
keep_prob = tf.placeholder(tf.float32)
tf_X = tf.placeholder(tf.float32,[None,6,6,1])
tf_Y = tf.placeholder(tf.float32,[None,5])

# 卷积层+激活层 
conv_filter_w1 = tf.Variable(tf.random_normal([3, 3, 1, 5])) 
conv_filter_b1 = tf.Variable(tf.random_normal([5])) 
relu_feature_maps1 = tf.nn.relu(tf.nn.conv2d(tf_X, conv_filter_w1, strides=[1, 1, 1, 1], padding='SAME') + conv_filter_b1)

# 池化层
max_pool1 = tf.nn.max_pool(relu_feature_maps1, ksize = [1,3,3,1], strides = [1,2,2,1], padding = 'SAME')

# 卷积层 
conv_filter_w2 = tf.Variable(tf.random_normal([3, 3, 5, 5])) 
conv_filter_b2 = tf.Variable(tf.random_normal([5])) 
conv_out2 = tf.nn.conv2d(relu_feature_maps1, conv_filter_w2, strides = [1, 2, 2, 1], padding = 'SAME') + conv_filter_b2 

# BN归一化层+激活层 
batch_mean, batch_var = tf.nn.moments(conv_out2, [0, 1, 2], keep_dims = True) 
shift = tf.Variable(tf.zeros([5])) 
scale = tf.Variable(tf.ones([5])) 
epsilon = 1e-3 
BN_out = tf.nn.batch_normalization(conv_out2, batch_mean, batch_var, shift, scale, epsilon) 
relu_BN_maps2 = tf.nn.relu(BN_out)

# 池化层
max_pool2 = tf.nn.max_pool(relu_BN_maps2, ksize = [1,3,3,1], strides = [1,2,2,1], padding = 'SAME')

# 将特征图进行展开
max_pool2_flat = tf.reshape(max_pool2, [-1, 2*2*5])

# 全连接层 
fc_w1 = tf.Variable(tf.random_normal([2*2*5,50])) 
fc_b1 = tf.Variable(tf.random_normal([50])) 
fc_out1 = tf.nn.relu(tf.matmul(max_pool2_flat, fc_w1) + fc_b1)
fc_out1 = tf.nn.dropout(fc_out1, keep_prob)

# 输出层 
out_w1 = tf.Variable(tf.random_normal([50,5])) 
out_b1 = tf.Variable(tf.random_normal([5])) 
pred = tf.nn.softmax(tf.matmul(fc_out1, out_w1)+out_b1)

loss = -1*tf.reduce_mean(tf_Y*tf.log(tf.clip_by_value(pred, 1e-11, 1.0)))
train_step = tf.train.AdamOptimizer(1e-3).minimize(loss)

y_pred = tf.argmax(pred, 1)
bool_pred = tf.equal(tf.argmax(tf_Y,1), y_pred)
# 准确率
accuracy = tf.reduce_mean(tf.cast(bool_pred, tf.float32))

In [6]:
print("######### The train & test process of SCORE1 ##########")
with tf.Session() as sess: 
    sess.run(tf.global_variables_initializer()) 
    for epoch in range(1000): 
        # 迭代1000个周期 
        for batch_xs,batch_ys in generatebatch(X1_train, y1_train, y1_train.shape[0], batch_size): 
            # 每个周期进行MBGD算法 
            sess.run(train_step, feed_dict={tf_X:batch_xs, tf_Y:batch_ys, keep_prob: 0.5}) 
        if(epoch%100==0): 
            res = sess.run(accuracy,feed_dict={tf_X:X1_train, tf_Y:y1_train, keep_prob: 0.5}) 
            #print((epoch,res))
            print("The %d epoch, the accuracy is %f " %(epoch, res))
    res_ypred = y_pred.eval(feed_dict={tf_X:X1_test, tf_Y:y1_test_code, keep_prob: 1}).flatten() 
    # 只能预测一批样本，不能预测一个样本 
    print('The result of the test bath is  ', res_ypred)
print("The accurate score of the test bath is  ", accuracy_score(y1_test, res_ypred.reshape(-1, 1)))
print("The MSE of the test bach is  ",  mean_squared_error(y1_test, res_ypred.reshape(-1, 1)))

print("######### The train & test process of SCORE2 ##########")
with tf.Session() as sess: 
    sess.run(tf.global_variables_initializer()) 
    for epoch in range(1000): 
        # 迭代1000个周期 
        for batch_xs,batch_ys in generatebatch(X2_train, y2_train, y2_train.shape[0], batch_size): 
            # 每个周期进行MBGD算法 
            sess.run(train_step, feed_dict={tf_X:batch_xs, tf_Y:batch_ys, keep_prob: 0.5}) 
        if(epoch%100==0): 
            res = sess.run(accuracy,feed_dict={tf_X:X2_train, tf_Y:y2_train, keep_prob: 1}) 
            #print((epoch,res))
            print("The %d epoch, the accuracy is %f " %(epoch, res))
    res_ypred = y_pred.eval(feed_dict={tf_X:X2_test, tf_Y:y2_test_code, keep_prob: 1}).flatten() 
    # 只能预测一批样本，不能预测一个样本 
    print('The result of the test bath is  ', res_ypred)
print("The accurate score of the test bath is  ", accuracy_score(y2_test, res_ypred.reshape(-1, 1)))
print("The MSE of the test bach is  ",  mean_squared_error(y2_test, res_ypred.reshape(-1, 1)))

print("######### The train & test process of SCORE3 ##########")
with tf.Session() as sess: 
    sess.run(tf.global_variables_initializer()) 
    for epoch in range(1000): 
        # 迭代1000个周期 
        for batch_xs,batch_ys in generatebatch(X3_train, y3_train, y3_train.shape[0], batch_size): 
            # 每个周期进行MBGD算法 
            sess.run(train_step, feed_dict={tf_X:batch_xs, tf_Y:batch_ys, keep_prob: 0.5}) 
        if(epoch%100==0): 
            res = sess.run(accuracy,feed_dict={tf_X:X3_train, tf_Y:y3_train, keep_prob: 1}) 
            #print((epoch,res))
            print("The %d epoch, the accuracy is %f " %(epoch, res))
    res_ypred = y_pred.eval(feed_dict={tf_X:X3_test, tf_Y:y3_test_code, keep_prob: 1}).flatten() 
    # 只能预测一批样本，不能预测一个样本 
    print('The accurate of the train bath is  ', res_ypred)
print("The accurate score of the test bath is  ", accuracy_score(y3_test, res_ypred.reshape(-1, 1)))
print("The MSE of the test bach is  ",  mean_squared_error(y3_test, res_ypred.reshape(-1, 1)))

print("######### The train & test process of SCORE4 ##########")
with tf.Session() as sess: 
    sess.run(tf.global_variables_initializer()) 
    for epoch in range(1000): 
        # 迭代1000个周期 
        for batch_xs,batch_ys in generatebatch(X4_train, y4_train, y4_train.shape[0], batch_size): 
            # 每个周期进行MBGD算法 
            sess.run(train_step, feed_dict={tf_X:batch_xs, tf_Y:batch_ys, keep_prob: 0.5}) 
        if(epoch%100==0): 
            res = sess.run(accuracy,feed_dict={tf_X:X4_train, tf_Y:y4_train, keep_prob: 1}) 
            #print((epoch,res))
            print("The %d epoch, the accuracy is %f " %(epoch, res))
    res_ypred = y_pred.eval(feed_dict={tf_X:X4_test, tf_Y:y4_test_code, keep_prob: 1}).flatten() 
    # 只能预测一批样本，不能预测一个样本 
    print('The result of the test bath is  ', res_ypred)
print("The accurate score of the test bath is  ", accuracy_score(y4_test, res_ypred.reshape(-1, 1)))
print("The MSE of the test bach is  ",  mean_squared_error(y4_test, res_ypred.reshape(-1, 1)))

######### The train & test process of SCORE1 ##########
The 0 epoch, the accuracy is 0.145403 
The 100 epoch, the accuracy is 0.763593 
The 200 epoch, the accuracy is 0.763647 
The 300 epoch, the accuracy is 0.763620 
The 400 epoch, the accuracy is 0.763863 
The 500 epoch, the accuracy is 0.764133 
The 600 epoch, the accuracy is 0.763782 
The 700 epoch, the accuracy is 0.763782 
The 800 epoch, the accuracy is 0.763890 
The 900 epoch, the accuracy is 0.763971 
The result of the test bath is   [4 4 4 ..., 4 4 4]
The accurate score of the test bath is   0.14439585358
The MSE of the test bach is   0.890022675737
######### The train & test process of SCORE2 ##########
The 0 epoch, the accuracy is 0.758355 
The 100 epoch, the accuracy is 0.758355 
The 200 epoch, the accuracy is 0.758436 
The 300 epoch, the accuracy is 0.758328 
The 400 epoch, the accuracy is 0.758382 
The 500 epoch, the accuracy is 0.758436 
The 600 epoch, the accuracy is 0.758517 
The 700 epoch, the accuracy is 0.758463 
Th

In [ ]:
from collections import Counter
Counter(res_ypred.tolist())